<a href="https://colab.research.google.com/github/flydragon2018/colab_notebooks/blob/main/hw_mynet_effnetb0_grainsize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Sep 12 17:40:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#! unzip -q  /content/drive/MyDrive/hw/2021hwjx-data-train.zip -d /content/

In [2]:
!unzip -q /content/drive/MyDrive/hw/goldsize/cleaned_resnet50_data.zip  -d / 

In [3]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=80ecd0b96ac64f5634e4d8bf20c877c5503839f90a52d7e637392a05a1d1369b
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [4]:
%%writefile utils.py
from collections import defaultdict, deque
import datetime
import time
import torch
import torch.distributed as dist
import numpy as np
import errno
import os


class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)


class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object has no attribute '{}'".format(
            type(self).__name__, attr))

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(
                "{}: {}".format(name, str(meter))
            )
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt='{avg:.4f}')
        data_time = SmoothedValue(fmt='{avg:.4f}')
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        if torch.cuda.is_available():
            log_msg = self.delimiter.join([
                header,
                '[{0' + space_fmt + '}/{1}]',
                'eta: {eta}',
                '{meters}',
                'time: {time}',
                'data: {data}',
                'max mem: {memory:.0f}'
            ])
        else:
            log_msg = self.delimiter.join([
                header,
                '[{0' + space_fmt + '}/{1}]',
                'eta: {eta}',
                '{meters}',
                'time: {time}',
                'data: {data}'
            ])
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                if torch.cuda.is_available():
                    print(log_msg.format(
                        i, len(iterable), eta=eta_string,
                        meters=str(self),
                        time=str(iter_time), data=str(data_time),
                        memory=torch.cuda.max_memory_allocated() / MB))
                else:
                    print(log_msg.format(
                        i, len(iterable), eta=eta_string,
                        meters=str(self),
                        time=str(iter_time), data=str(data_time)))
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print('{} Total time: {}'.format(header, total_time_str))

def accuracy(output, target, idx_to_class):
    """Computes the accuracy """
    with torch.no_grad():
        batch_size = target.size(0)
        _, pred = output.topk(1, 1, True, True)
        pred = pred.t()
        pred = pred.flatten().cpu().numpy()
        gt = target.cpu().numpy()

        # idx maps to label
        pre_label = torch.tensor([float(idx_to_class[pred[i]]) for i in range(len(pred))])
        gt_label = torch.tensor([float(idx_to_class[gt[i]]) for i in range(len(gt))])

        # compute acc-0 and acc-0.5
        correct_0 = torch.le(abs(pre_label - gt_label), 0)
        correct_0_5 = torch.le(abs(pre_label - gt_label), 0.5)
        acc0 = correct_0.sum() * (100.0 / batch_size)
        acc0_5 = correct_0_5.sum() * (100.0 / batch_size)

    return acc0, acc0_5


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__
    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ['WORLD_SIZE'])
        args.gpu = int(os.environ['LOCAL_RANK'])
    elif 'SLURM_PROCID' in os.environ:
        args.rank = int(os.environ['SLURM_PROCID'])
        args.gpu = args.rank % torch.cuda.device_count()
    elif hasattr(args, "rank"):
        pass
    else:
        print('Not using distributed mode')
        args.distributed = False
        return

    args.distributed = True
    torch.cuda.set_device(args.gpu)
    args.dist_backend = 'nccl'
    print('| distributed init (rank {}): {}'.format(
        args.rank, args.dist_url), flush=True)
    torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                         world_size=args.world_size, rank=args.rank)
    setup_for_distributed(args.rank == 0)


Writing utils.py


In [5]:
#model = models.alexnet(pretrained=True)
#model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [6]:
%%writefile  mynet_train.py
import os
import argparse



#import moxing as mox
import time
import torch
from torch import nn
import torchvision
from torchvision import transforms
import torchvision.models as models
import utils

from efficientnet_pytorch import EfficientNet

from sklearn.model_selection import KFold

"""
消除随机因素的影响
"""
torch.manual_seed(2021)


def parse_args():
    parser = argparse.ArgumentParser(description='Train a classification algorithm')
    parser.add_argument('--data_url', type=str, required=True,
                        help='the dataset dir of dataset')
    parser.add_argument('--train_url', type=str, required=True,
                        help='the checkpoint dir obs')
    parser.add_argument('--init_method', type=str, required=False,
                        help='')
    parser.add_argument('--num_gpus', type=int, required=False, default=1,
                        help='')
    parser.add_argument('--last_path', type=str, required=False,
                        help='')
    parser.add_argument('--train-dir', type=str, required=True,
                        help='the dataset dir of training dataset')
    parser.add_argument('--validate-dir', type=str, required=False,
                        default=None,
                        help='the dataset dir of validation dataset')
    parser.add_argument('--ckpt-dir', type=str, required=True,
                        help='the checkpoint dir')
    parser.add_argument('--num-classes', type=int, required=True,
                        help='num-classes, do not include bg')
    parser.add_argument('--batch-size', type=int, required=True,
                        help='batch size')
    parser.add_argument('--num-epochs', type=int, required=True,
                        help='the number of epochs')
    args = parser.parse_args()
    return args


def prepare_data(data_url, training_dir, validate_dir):
    print(data_url)
    # copy training dataset from obs data_url
    if not os.path.exists(training_dir):
        os.makedirs(training_dir)
        print('mkdir: {}'.format(training_dir))
    
    os.system('ls {}'.format(training_dir))
    os.system('cd {} && unzip -qq 2021hwjx-data-train.zip'.format(
        training_dir))

    # copy validate dataset from obs data_url if validate_dir is not None
    if validate_dir is not None:
        print('process validate_dir')
        if not os.path.exists(validate_dir):
            os.makedirs(validate_dir)
            print('mkdir: {}'.format(validate_dir))
       
        os.system('ls {}'.format(validate_dir))
        os.system('cd {} && unzip -qq 2021hwjx-data-val.zip'.format(
            validate_dir))



def load_data(dir, is_train):
    print("Loading data")
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    if is_train:
        dataset = torchvision.datasets.ImageFolder(
            dir,
            transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ]))
        sampler = torch.utils.data.RandomSampler(dataset)
    else:
        dataset = torchvision.datasets.ImageFolder(
            dir,
            transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ColorJitter(contrast=(1)),
                transforms.ToTensor(),
                normalize,
            ]))
        sampler = torch.utils.data.SequentialSampler(dataset)

    return dataset, sampler

'''
def initial_model(class_num):
    # load an classification model pre-trained on Imagenet
    #model = torchvision.models.__dict__['resnet50'](pretrained=True)
    #model = EfficientNet.from_pretrained('efficientnet-b7')
    model = models.vgg16_bn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.classifier[6].in_features 
    model.classifier[6]=nn.Linear(in_features, class_num)
     
    return model
'''
def initial_model(class_num):
    # load an classification model pre-trained on Imagenet
    #model = torchvision.models.__dict__['resnet50'](pretrained=True)
    model = EfficientNet.from_pretrained('efficientnet-b0')
    # get number of input features for the classifier
    channel_in = model._fc.in_features
    model._fc = nn.Linear(channel_in, class_num)
    return model

def train_one_epoch(model, criterion, optimizer, data_loader, idx_to_class, device, epoch, print_freq):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value}'))
    metric_logger.add_meter('img/s', utils.SmoothedValue(window_size=10, fmt='{value}'))

    header = 'Epoch: [{}]'.format(epoch)
    for image, target in metric_logger.log_every(data_loader, print_freq, header):
        start_time = time.time()
        image, target = image.to(device), target.to(device)
        output = model(image)
        loss = criterion(output, target)

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        acc0, acc0_5 = utils.accuracy(output, target, idx_to_class)
        acc = 0.5 * acc0 + 0.5 * acc0_5
        batch_size = image.shape[0]
        metric_logger.update(loss=loss.item(), lr=optimizer.param_groups[0]["lr"])
        metric_logger.meters['acc0'].update(acc0.item(), n=batch_size)
        metric_logger.meters['acc0_5'].update(acc0_5.item(), n=batch_size)
        metric_logger.meters['acc'].update(acc.item(), n=batch_size)
        metric_logger.meters['img/s'].update(batch_size / (time.time() - start_time))
    print(' *Train acc@0 {top0.global_avg:.3f} Train acc@0.5 {top5.global_avg:.3f} Train acc@ {top.global_avg:.3f}'
          .format(top0=metric_logger.acc0, top5=metric_logger.acc0_5, top=metric_logger.acc))


def evaluate(model, criterion, data_loader, idx_to_class, device, print_freq=100):
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'
    with torch.no_grad():
        for image, target in metric_logger.log_every(data_loader, print_freq, header):
            image = image.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)
            output = model(image)
            loss = criterion(output, target)

            acc0, acc0_5 = utils.accuracy(output, target, idx_to_class)
            acc = 0.5 * acc0 + 0.5 * acc0_5
            # could have been padded in distributed setup
            batch_size = image.shape[0]
            metric_logger.update(loss=loss.item())
            metric_logger.meters['acc0'].update(acc0.item(), n=batch_size)
            metric_logger.meters['acc0_5'].update(acc0_5.item(), n=batch_size)
            metric_logger.meters['acc'].update(acc.item(), n=batch_size)
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()

    print(' *Eval acc@0 {top0.global_avg:.3f} Eval acc@0.5 {top5.global_avg:.3f} Eval acc@ {top.global_avg:.3f}'
          .format(top0=metric_logger.acc0, top5=metric_logger.acc0_5, top=metric_logger.acc))
    return metric_logger.acc0.global_avg


def main(args):
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if \
        torch.cuda.is_available() else torch.device('cpu')

    train_dir = os.path.join(args.train_dir, 'train')
    dataset, train_sampler = load_data(train_dir, is_train=True)
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size,
        sampler=train_sampler, num_workers=4, pin_memory=True)
    # provide idx and class relationgships
    idx_to_class = {value: key for key, value in dataset.class_to_idx.items()}
	
	
	

    if args.validate_dir is not None:
        val_dir = os.path.join(args.validate_dir, 'val')
        dataset_test, test_sampler = load_data(val_dir, is_train=False)
        data_loader_test = torch.utils.data.DataLoader(
            dataset_test, batch_size=args.batch_size,
            sampler=test_sampler, num_workers=4, pin_memory=True)

    # get the model using our helper function
    model = initial_model(args.num_classes)

    # move model to the right device
    model.to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.SGD(
        model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

    print("Start training")
    for epoch in range(args.num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, criterion, optimizer, data_loader, idx_to_class, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()

        # evaluate on the test dataset
        if args.validate_dir is not None:
            evaluate(model, criterion, data_loader_test, idx_to_class, device=device)
        checkpoint = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'epoch': epoch,
            'args': args}

        if args.ckpt_dir is not None:
            if not os.path.exists(args.ckpt_dir):
                os.makedirs(args.ckpt_dir)
                print('mkdir: {}'.format(args.ckpt_dir))
        local_ckpt_path = os.path.join(args.ckpt_dir,
                                       'model_{}.pth'.format(epoch))

        # write classes.txt
        local_classes_path = os.path.join(args.ckpt_dir, 'classes.txt')
        with open(local_classes_path, 'w') as f:
            for i in range(len(idx_to_class)):
                f.write(idx_to_class[i] + "\n")

        # save model every 5 steps and upload model to obs
        if epoch % 10 == 0:
            utils.save_on_master(checkpoint, local_ckpt_path)

            if args.train_url is not None:
                # obs://obs-2021hwjx-baseline/data
                obs_target_path = os.path.join('{}'.format(args.train_url),
                                           os.path.basename(local_ckpt_path))
                # OBS 路径每隔30s会将内容同步到线上OBS桶中
                os.system("cp {} {}".format(local_ckpt_path, obs_target_path))
                print('finish upload {}->{}'.format(local_ckpt_path,
                                                obs_target_path))

 




if __name__ == "__main__":
    args = parse_args()
    prepare_data(
        args.data_url,
        args.train_dir,
        args.validate_dir
    )
    print(args.data_url)
    print(args.train_url)
    os.system('ls {}'.format(args.train_url))
    main(args)


Writing mynet_train.py


In [11]:
!python /content/mynet_train.py --data_url /content/cleaned_data/train  --train_url  /content/cleaned_data   --train-dir  /content/cleaned_data    --ckpt-dir /content/  --num-classes 14 --batch-size  64 --num-epochs 120

/content/cleaned_data/train
train
unzip:  cannot find or open 2021hwjx-data-train.zip, 2021hwjx-data-train.zip.zip or 2021hwjx-data-train.zip.ZIP.
/content/cleaned_data/train
/content/cleaned_data
train
Loading data
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100% 20.4M/20.4M [00:00<00:00, 91.6MB/s]
Loaded pretrained weights for efficientnet-b0
Start training
Epoch: [0]  [ 0/58]  eta: 0:07:40  lr: 0.1 

cp: cannot stat '/content/model_100.pth': No such file or directory


In [12]:
! cp /content/model_90.pth  /content/drive/MyDrive/hw/goldsize/model_b0_90_range_cleaned.pth